In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

In [2]:
import torch
import math


dtype = torch.float
device = torch.device("cpu")

In [3]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [4]:
 # Data load

df = pd.read_csv('경기도_clustered1.csv',encoding = 'cp949')

cluster = pd.read_csv('경기도_clustered1.csv',encoding = 'cp949',index_col = 0)
cluster

,수원시,성남시,의정부시,안양시,부천시,광명시,평택시,동두천시,안산시,고양시,...,이천시,안성시,김포시,화성시,광주시,양주시,여주시,포천시,의왕시,오산시
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,63.4,67.8,77.7,60.5,68.1,53.0,92.6,92.6,66.8,69.5,...,89.2,92.6,71.1,73.0,82.4,82.0,102.8,93.6,59.7,83.2
2013-02-01,63.3,67.2,77.4,60.2,67.7,52.9,92.5,90.9,66.6,69.0,...,89.4,92.8,70.9,72.9,82.0,81.7,102.8,93.6,59.1,83.0
2013-03-01,63.2,66.9,76.9,60.0,67.4,52.6,92.4,90.1,66.3,68.6,...,89.7,92.8,70.8,72.8,81.9,81.4,102.7,93.6,58.7,82.9
2013-04-01,63.3,67.1,76.4,59.8,67.4,52.6,92.4,90.0,66.3,68.1,...,90.1,93.5,70.7,73.2,82.3,81.7,102.7,93.9,58.6,82.9
2013-05-01,63.4,67.5,76.4,59.6,67.6,52.9,92.4,89.8,66.6,68.1,...,90.4,93.8,71.0,73.6,82.3,81.9,103.6,94.7,58.8,82.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-01,109.5,104.8,107.2,111.7,110.2,107.3,113.2,107.7,110.7,107.6,...,110.3,114.8,105.3,111.6,107.9,109.5,106.6,108.4,112.9,115.9
2021-12-01,109.8,105.1,107.4,112.0,110.8,107.4,114.1,107.5,111.2,108.0,...,112.3,116.1,105.6,111.6,108.4,109.8,107.2,109.0,113.0,116.7
2022-01-01,109.8,105.1,107.4,111.8,110.8,107.3,114.3,107.5,111.3,108.1,...,113.3,116.9,105.6,111.5,108.7,109.8,107.1,109.2,113.0,116.8


In [5]:
data = pd.read_csv('경기도변수.csv')
feature = pd.read_csv('경기도변수.csv',index_col = 0)

data

,date,seoul,Avg_1,Avg_2,Avg_3,MMI,CPI,대출금리,전세가격지수 전국,전세가격지수 경기도,환율,M2,코스피,코스닥,건축허용면적,거래량 전국,거래량 경기
0,2013-01-01,73.5,64.75625,93.900000,80.1,102.868,92.728,5.78,82.689331,76.007636,1089.0,1841.1,1961.90,503.67,8413721.0,54632,13631
1,2013-02-01,73.2,64.42500,93.666667,79.8,90.009,93.038,5.66,82.871183,76.108044,1083.0,1857.1,2026.50,535.87,7268443.0,65640,14925
2,2013-03-01,73.1,64.22500,93.550000,79.6,101.388,92.952,5.54,83.162722,76.376687,1111.0,1862.4,2004.90,555.02,10160177.0,87563,17718
3,2013-04-01,73.2,64.30000,93.766667,79.5,101.295,92.823,5.42,83.554044,76.858189,1101.0,1867.7,1964.00,563.87,9757932.0,106498,22790
4,2013-05-01,73.4,64.49375,94.116667,79.6,102.313,92.823,5.24,83.797077,77.090528,1129.7,1870.3,2001.05,577.87,10837979.0,120393,25910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2021-10-01,103.5,107.45625,108.650000,108.8,114.800,103.350,3.47,102.450935,103.903947,1168.6,3543.4,2970.68,992.33,14689855.0,119686,29187
106,2021-11-01,104.1,108.45625,110.166667,109.8,119.300,103.870,3.58,102.917900,104.543227,1187.9,3594.7,2839.01,965.63,14483042.0,111172,26593
107,2021-12-01,104.4,108.79375,111.033333,110.2,126.700,104.040,3.68,103.178314,104.798091,1188.8,3620.1,2977.65,1033.98,19169150.0,101766,25669
108,2022-01-01,104.4,108.75000,111.383333,110.3,115.100,104.690,3.75,103.250891,104.790872,1205.5,3634.3,2663.34,872.87,13736304.0,87825,20544


In [10]:
x = cluster.replace(',','')



train_x = x.iloc[0:int(len(x)*Train_ratio),:]
train_x

,수원시,성남시,의정부시,안양시,부천시,광명시,평택시,동두천시,안산시,고양시,...,이천시,안성시,김포시,화성시,광주시,양주시,여주시,포천시,의왕시,오산시
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,63.4,67.8,77.7,60.5,68.1,53.0,92.6,92.6,66.8,69.5,...,89.2,92.6,71.1,73.0,82.4,82.0,102.8,93.6,59.7,83.2
2013-02-01,63.3,67.2,77.4,60.2,67.7,52.9,92.5,90.9,66.6,69.0,...,89.4,92.8,70.9,72.9,82.0,81.7,102.8,93.6,59.1,83.0
2013-03-01,63.2,66.9,76.9,60.0,67.4,52.6,92.4,90.1,66.3,68.6,...,89.7,92.8,70.8,72.8,81.9,81.4,102.7,93.6,58.7,82.9
2013-04-01,63.3,67.1,76.4,59.8,67.4,52.6,92.4,90.0,66.3,68.1,...,90.1,93.5,70.7,73.2,82.3,81.7,102.7,93.9,58.6,82.9
2013-05-01,63.4,67.5,76.4,59.6,67.6,52.9,92.4,89.8,66.6,68.1,...,90.4,93.8,71.0,73.6,82.3,81.9,103.6,94.7,58.8,82.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,73.9,85.3,82.5,74.7,79.0,76.3,90.4,92.2,70.6,80.5,...,100.9,94.2,85.3,80.5,84.5,87.1,106.7,97.4,72.0,80.5
2019-02-01,73.9,84.7,82.5,74.5,79.0,75.7,90.2,92.1,70.3,80.4,...,100.4,94.0,85.2,80.4,84.5,87.1,106.5,97.3,71.8,80.3
2019-03-01,73.8,84.1,82.4,74.3,78.9,74.5,89.9,92.0,70.1,80.3,...,99.7,93.8,85.1,80.1,84.4,87.0,105.9,97.1,71.1,80.0


In [49]:
train = data.copy()

In [50]:
train.corr()

,seoul,Avg_1,Avg_2,Avg_3,MMI,CPI,대출금리,전세가격지수 전국,전세가격지수 경기도,환율,M2,코스피,코스닥,건축허용면적,거래량 전국,거래량 경기
seoul,1.00000,0.90937,0.22791,0.77387,0.65565,0.98536,-0.78876,0.82194,0.83350,0.51119,0.98395,0.71342,0.80426,0.12069,0.27725,0.46107
Avg_1,0.90937,1.00000,0.43014,0.95232,0.65904,0.91431,-0.73825,0.85927,0.86416,0.45403,0.95073,0.82543,0.85191,0.21349,0.14592,0.22341
Avg_2,0.22791,0.43014,1.00000,0.63663,0.26381,0.31619,-0.26385,0.62986,0.60502,0.12227,0.25843,0.40436,0.41053,0.46749,-0.12229,-0.11360
Avg_3,0.77387,0.95232,0.63663,1.00000,0.61770,0.80261,-0.59546,0.82992,0.82501,0.35248,0.82588,0.83377,0.80558,0.25581,-0.00010,0.04684
MMI,0.65565,0.65904,0.26381,0.61770,1.00000,0.64364,-0.50888,0.59033,0.59217,0.22583,0.66092,0.59047,0.57564,0.37032,0.31797,0.35594
CPI,0.98536,0.91431,0.31619,0.80261,0.64364,1.00000,-0.78866,0.86051,0.86782,0.47790,0.97765,0.73679,0.80020,0.13549,0.25272,0.43478
대출금리,-0.78876,-0.73825,-0.26385,-0.59546,-0.50888,-0.78866,1.00000,-0.88344,-0.89777,-0.55747,-0.82740,-0.57529,-0.73038,-0.41693,-0.54202,-0.61100
전세가격지수 전국,0.82194,0.85927,0.62986,0.82992,0.59033,0.86051,-0.88344,1.00000,0.99877,0.44409,0.85400,0.73851,0.83238,0.45508,0.31973,0.41120
전세가격지수 경기도,0.83350,0.86416,0.60502,0.82501,0.59217,0.86782,-0.89777,0.99877,1.00000,0.46612,0.86609,0.73232,0.83338,0.44888,0.33749,0.42914
환율,0.51119,0.45403,0.12227,0.35248,0.22583,0.47790,-0.55747,0.44409,0.46612,1.00000,0.52514,0.03280,0.22826,0.16604,0.12524,0.22564


In [9]:
# Training/Test ratio = 7:3

Train_ratio = 0.7

Test_ratio = 0.3

In [52]:
result =  pd.DataFrame(data=[0,0,0,0],
                             index = ['MAE','RMSE','MAPE', 'R2'],
                             columns=['test'])
predict =  pd.DataFrame(data=[0]*6,columns=['test'],index = predicted_index  )
predict

,test
2022-01-01,0
2022-02-01,0
2022-03-01,0
2022-04-01,0
2022-05-01,0
2022-06-01,0


lag 변수 생성

In [53]:
df.set_index('date', inplace=True)
data.set_index('date', inplace=True)

In [54]:
lag_col= list(df.columns)


lag_amount = 3

for col in lag_col:
    for i in range(lag_amount):
        df['{0}_lag{1}'.format(col,i+1)] = df['{}'.format(col)].shift(i+1)
        
df.dropna(inplace=True)

In [55]:
df = df.iloc[:-3,:]
df

,수원시,성남시,의정부시,안양시,부천시,광명시,평택시,동두천시,안산시,고양시,...,여주시_lag3,포천시_lag1,포천시_lag2,포천시_lag3,의왕시_lag1,의왕시_lag2,의왕시_lag3,오산시_lag1,오산시_lag2,오산시_lag3
date,,,,,,,,,,,,,,,,,,,,,
2013-04-01,63.30000,67.10000,76.40000,59.80000,67.40000,52.60000,92.40000,90.00000,66.30000,68.10000,...,102.80000,93.60000,93.60000,93.60000,58.70000,59.10000,59.70000,82.90000,83.00000,83.20000
2013-05-01,63.40000,67.50000,76.40000,59.60000,67.60000,52.90000,92.40000,89.80000,66.60000,68.10000,...,102.80000,93.90000,93.60000,93.60000,58.60000,58.70000,59.10000,82.90000,82.90000,83.00000
2013-06-01,63.50000,67.90000,76.20000,59.50000,67.60000,53.00000,92.60000,89.60000,66.80000,68.00000,...,102.70000,94.70000,93.90000,93.60000,58.80000,58.60000,58.70000,82.90000,82.90000,82.90000
2013-07-01,63.40000,67.70000,75.50000,59.50000,67.50000,53.10000,92.50000,89.30000,66.70000,67.90000,...,102.70000,95.10000,94.70000,93.90000,59.10000,58.80000,58.60000,82.40000,82.90000,82.90000
2013-08-01,63.20000,67.40000,75.20000,59.40000,67.20000,52.90000,92.30000,89.00000,66.50000,67.90000,...,103.60000,94.70000,95.10000,94.70000,58.70000,59.10000,58.80000,82.10000,82.40000,82.90000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,104.70000,102.60000,103.80000,106.20000,104.50000,104.00000,106.20000,104.50000,105.00000,103.10000,...,99.10000,101.60000,100.00000,97.90000,103.10000,100.00000,96.30000,103.50000,100.00000,96.00000
2021-09-01,106.50000,103.60000,105.20000,108.30000,106.70000,105.50000,108.80000,106.40000,107.10000,105.00000,...,100.00000,103.70000,101.60000,100.00000,106.40000,103.10000,100.00000,107.30000,103.50000,100.00000
2021-10-01,108.40000,104.30000,106.50000,110.50000,108.70000,106.70000,111.70000,107.30000,109.30000,106.70000,...,101.10000,105.80000,103.70000,101.60000,109.10000,106.40000,103.10000,110.70000,107.30000,103.50000


In [56]:
lag_col= list(data.columns)


lag_amount = 3

for col in lag_col:
    for i in range(lag_amount):
        data['{0}_lag{1}'.format(col,i+1)] = data['{}'.format(col)].shift(i+1)
        
data.dropna(inplace=True)

In [57]:
data= data.iloc[:-2,:]
data

,seoul,Avg_1,Avg_2,Avg_3,MMI,CPI,대출금리,전세가격지수 전국,전세가격지수 경기도,환율,...,코스닥_lag3,건축허용면적_lag1,건축허용면적_lag2,건축허용면적_lag3,거래량 전국_lag1,거래량 전국_lag2,거래량 전국_lag3,거래량 경기_lag1,거래량 경기_lag2,거래량 경기_lag3
date,,,,,,,,,,,,,,,,,,,,,
2013-04-01,73.20000,64.30000,93.76667,79.50000,101.29500,92.82300,5.42000,83.55404,76.85819,1101.00000,...,503.67000,10160177.00000,7268443.00000,8413721.00000,87563.00000,65640.00000,54632.00000,17718.00000,14925.00000,13631.00000
2013-05-01,73.40000,64.49375,94.11667,79.60000,102.31300,92.82300,5.24000,83.79708,77.09053,1129.70000,...,535.87000,9757932.00000,10160177.00000,7268443.00000,106498.00000,87563.00000,65640.00000,22790.00000,17718.00000,14925.00000
2013-06-01,73.20000,64.55000,94.30000,79.50000,99.72200,92.71000,5.05000,83.97808,77.26569,1142.00000,...,555.02000,10837979.00000,9757932.00000,10160177.00000,120393.00000,106498.00000,87563.00000,25910.00000,22790.00000,17718.00000
2013-07-01,72.80000,64.36875,94.23333,79.10000,99.90700,92.90900,5.09000,84.23058,77.47430,1123.50000,...,563.87000,10007009.00000,10837979.00000,9757932.00000,160468.00000,120393.00000,106498.00000,33741.00000,25910.00000,22790.00000
2013-08-01,72.50000,64.17500,94.08333,78.80000,96.11500,93.23800,5.06000,84.58286,77.93904,1110.00000,...,577.87000,11233555.00000,10007009.00000,10837979.00000,74944.00000,160468.00000,120393.00000,16437.00000,33741.00000,25910.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-01,101.70000,104.06250,104.50000,104.70000,110.50000,102.75000,3.32000,101.22584,101.99146,1159.50000,...,981.78000,14591577.00000,16013705.00000,13891755.00000,140102.00000,144759.00000,154949.00000,39146.00000,38191.00000,42027.00000
2021-09-01,102.70000,105.82500,106.61667,106.90000,111.10000,103.17000,3.40000,101.82469,102.96483,1184.00000,...,1029.96000,13946424.00000,14591577.00000,16013705.00000,133495.00000,140102.00000,144759.00000,36154.00000,39146.00000,38191.00000
2021-10-01,103.50000,107.45625,108.65000,108.80000,114.80000,103.35000,3.47000,102.45093,103.90395,1168.60000,...,1031.14000,14652194.00000,13946424.00000,14591577.00000,122333.00000,133495.00000,140102.00000,33662.00000,36154.00000,39146.00000


In [58]:
for i in range(5,10):
    clust_n = int(cluster[cluster.columns[i]][-1])
    
    answer = []
    for num in range(11):
        if abs(train.corr()['Avg_{}'.format(clust_n+1)][num])>=0.7:
            answer.append(num)
    
    col =[]
    for j in  answer:
        col.append(data.columns[j])
    
    
    x = data[col].replace(',','')
    y = df.iloc[:,i:i+1]
    
       
    train_x = x.iloc[0:int(len(x)*Train_ratio),:]

    train_y = y.iloc[0:int(len(x)*Train_ratio),:]

    test_x = x.iloc[int(len(x)*Train_ratio):,:]

    test_y = y.iloc[int(len(x)*Train_ratio):,:]
    

    
    

    # Normalizing , 둘다 학습하여 성능비교할것

    minmax = MinMaxScaler()

    standard = StandardScaler()

    
    yscaler = minmax.fit(y.values.reshape(-1,1))
    train_x = minmax.fit_transform(train_x)

    train_y = minmax.fit_transform(train_y)

    test_x = minmax.fit_transform(test_x)

    test_y = minmax.fit_transform(test_y)





    # Numpy array상태로는 학습이 불가능하므로, Torch Variable 형태로 변경(data/grad/grad_fn)

    train_x_tensor = Variable(torch.Tensor(train_x))

    train_y_tensor = Variable(torch.Tensor(train_y))

    

    test_x_tensor = Variable(torch.Tensor(test_x))

    test_y_tensor = Variable(torch.Tensor(test_y))

    

    train_x_tensor_final = torch.reshape(train_x_tensor, (train_x_tensor.shape[0], 1, train_x_tensor.shape[1]))

    train_y_tensor_final = torch.reshape(train_y_tensor, (train_y_tensor.shape[0], 1, train_y_tensor.shape[1]))

    test_x_tensor_final = torch.reshape(test_x_tensor, (test_x_tensor.shape[0], 1, test_x_tensor.shape[1]))

    test_y_tensor_final = torch.reshape(test_y_tensor,(test_y_tensor.shape[0], 1, test_y_tensor.shape[1]) )

    


    ## LSTM network modeling

    class LSTM_Jun(nn.Module):

        def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length) : 

            super(LSTM_Jun, self).__init__()

            self.num_classes = num_classes

            self.num_layers = num_layers

            self.input_size = input_size

            self.hidden_size = hidden_size

            self.seq_length = seq_length

            self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True)

            self.layer_1 = nn.Linear(hidden_size, 256)

            self.layer_2 = nn.Linear(256,256)

            self.layer_3 = nn.Linear(256,128)

            self.layer_out = nn.Linear(128, num_classes)

            self.relu = nn.ReLU() #Activation Func

        def forward(self,x):

            h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #Hidden State

            c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #Internal Process States



            output, (hn, cn) = self.lstm(x, (h_0, c_0))



            hn = hn.view(-1, self.hidden_size) # Reshaping the data for starting LSTM network

            out = self.relu(hn) #pre-processing for first layer

            out = self.layer_1(out) # first layer

            out = self.relu(out) # activation func relu

            out = self.layer_2(out)

            out = self.relu(out)

            out = self.layer_3(out)

            out = self.relu(out)

            out = self.layer_out(out) #Output layer

            return out

 

    # Code Main

    num_epochs = 100000
    learning_rate = 0.005

    input_size = int(len(x.columns))

    hidden_size = 2 # number of features in hidden state

    num_layers = 1

    num_classes = int(len(y.columns))



    LSTM_Jun = LSTM_Jun(num_classes, input_size, hidden_size, num_layers, train_x_tensor_final.shape[1]).to(device)



    loss_function = torch.nn.MSELoss()

    optimizer = torch.optim.Adam(LSTM_Jun.parameters(), lr = learning_rate)

 

    for epoch in range(num_epochs) : 

        outputs = LSTM_Jun.forward(train_x_tensor_final.to(device))

        optimizer.zero_grad()

        loss = loss_function(outputs, train_y_tensor.to(device))

        loss.backward()

        optimizer.step() # improve from loss = back propagation

        if epoch % 100 == 0 :

            print("Epoch : %d, loss : %1.5f" % (epoch, loss.item()))
            

    # Estimated Value

    test_predict = LSTM_Jun(test_x_tensor_final.to(device)) #Forward Pass

    predict_data = test_predict.data.detach().cpu().numpy() #numpy conversion

    predict_data = minmax.inverse_transform(predict_data) #inverse normalization(Min/Max)
    
    y_pred = predict_data[:,0]

    # Real Value

    real_data = test_y_tensor.data.numpy() # Real value

    real_data = minmax.inverse_transform(real_data) #inverse normalization 
    
    y_test = real_data[:,0]

    
    MAE = mean_absolute_error(y_test, y_pred)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)
    MAPE= np.mean(np.abs((y_test - y_pred) / y_test))
    pd.options.display.float_format = '{:.5f}'.format
    
    label = df.columns[i]
    Result = pd.DataFrame(data=[MAE,RMSE, MAPE,R2],
                         index = ['MAE','RMSE','MAPE','R2'],
                         columns=[label])
        
    

    result = pd.concat([result,Result],axis = 1)

    DAYS_TO_PREDICT = 6
    with torch.no_grad():
        test_seq = test_x_tensor_final[:-1]
        preds = []
        for i in range(DAYS_TO_PREDICT):
            y_test_pred = LSTM_Jun(test_seq)[-1:,0]
            pred = torch.flatten(y_test_pred).item()
            preds.append(pred)
            new_seq = test_seq.numpy().flatten()
            new_seq = np.append(new_seq, [pred])
            new_seq = new_seq[1:]
            test_seq = torch.as_tensor(new_seq).view(test_x_tensor_final[:-1].shape[0],1,test_x_tensor_final[:-1].shape[2]).float()

    # Prediction value 스케일링 역변환
    pred_values = yscaler.inverse_transform(np.array(preds).reshape(-1,1))
    pred_values

    # 예측값 반올림
    import math

    pred_values_ceiled = list(pred_values.flatten())
    predicted_cases=pred_values_ceiled
    predicted_cases

    predicted_index = pd.date_range(
      start=data.index[-1],
      periods=DAYS_TO_PREDICT + 1, freq = 'MS',
      closed='right'
    )

    predicted_index = pd.to_datetime(predicted_index, format='%Y%m%d')

    predicted_cases_i = pd.DataFrame(
      data=predicted_cases,
      index=predicted_index,
        columns = [label]
    )
    
    if not i :
        predict = predicted_cases_i
    else:
        predict =pd.concat([predict,predicted_cases_i],axis = 1)

    preds_i = pd.DataFrame(predicted_cases_i)
    feature.index = pd.to_datetime(feature.index)

    y_i = pd.DataFrame(data =df[df.columns[i]].values,index = df.index)
    y_i.index.name = ''

Epoch : 0, loss : 0.29319
Epoch : 100, loss : 0.00118
Epoch : 200, loss : 0.00336
Epoch : 300, loss : 0.00049
Epoch : 400, loss : 0.00044
Epoch : 500, loss : 0.00040
Epoch : 600, loss : 0.00019
Epoch : 700, loss : 0.00012
Epoch : 800, loss : 0.00013
Epoch : 900, loss : 0.00009
Epoch : 1000, loss : 0.00007
Epoch : 1100, loss : 0.00365
Epoch : 1200, loss : 0.00012
Epoch : 1300, loss : 0.00008
Epoch : 1400, loss : 0.00006
Epoch : 1500, loss : 0.00005
Epoch : 1600, loss : 0.00005
Epoch : 1700, loss : 0.00004
Epoch : 1800, loss : 0.00004
Epoch : 1900, loss : 0.00004
Epoch : 2000, loss : 0.00005
Epoch : 2100, loss : 0.00004
Epoch : 2200, loss : 0.00004
Epoch : 2300, loss : 0.00356
Epoch : 2400, loss : 0.00004
Epoch : 2500, loss : 0.00003
Epoch : 2600, loss : 0.00003
Epoch : 2700, loss : 0.00003
Epoch : 2800, loss : 0.00003
Epoch : 2900, loss : 0.00003
Epoch : 3000, loss : 0.00004
Epoch : 3100, loss : 0.00003
Epoch : 3200, loss : 0.00003
Epoch : 3300, loss : 0.00003
Epoch : 3400, loss : 0.000

Epoch : 27700, loss : 0.00000
Epoch : 27800, loss : 0.00000
Epoch : 27900, loss : 0.00000
Epoch : 28000, loss : 0.00004
Epoch : 28100, loss : 0.00000
Epoch : 28200, loss : 0.00000
Epoch : 28300, loss : 0.00000
Epoch : 28400, loss : 0.00000
Epoch : 28500, loss : 0.00000
Epoch : 28600, loss : 0.00000
Epoch : 28700, loss : 0.00000
Epoch : 28800, loss : 0.00000
Epoch : 28900, loss : 0.00000
Epoch : 29000, loss : 0.00000
Epoch : 29100, loss : 0.00000
Epoch : 29200, loss : 0.00003
Epoch : 29300, loss : 0.00000
Epoch : 29400, loss : 0.00000
Epoch : 29500, loss : 0.00000
Epoch : 29600, loss : 0.00000
Epoch : 29700, loss : 0.00000
Epoch : 29800, loss : 0.00000
Epoch : 29900, loss : 0.00000
Epoch : 30000, loss : 0.00000
Epoch : 30100, loss : 0.00000
Epoch : 30200, loss : 0.00000
Epoch : 30300, loss : 0.00000
Epoch : 30400, loss : 0.00000
Epoch : 30500, loss : 0.00000
Epoch : 30600, loss : 0.00000
Epoch : 30700, loss : 0.00000
Epoch : 30800, loss : 0.00009
Epoch : 30900, loss : 0.00000
Epoch : 31

Epoch : 55100, loss : 0.00000
Epoch : 55200, loss : 0.00002
Epoch : 55300, loss : 0.00000
Epoch : 55400, loss : 0.00000
Epoch : 55500, loss : 0.00000
Epoch : 55600, loss : 0.00000
Epoch : 55700, loss : 0.00000
Epoch : 55800, loss : 0.00000
Epoch : 55900, loss : 0.00000
Epoch : 56000, loss : 0.00000
Epoch : 56100, loss : 0.00000
Epoch : 56200, loss : 0.00000
Epoch : 56300, loss : 0.00000
Epoch : 56400, loss : 0.00007
Epoch : 56500, loss : 0.00000
Epoch : 56600, loss : 0.00000
Epoch : 56700, loss : 0.00000
Epoch : 56800, loss : 0.00000
Epoch : 56900, loss : 0.00000
Epoch : 57000, loss : 0.00000
Epoch : 57100, loss : 0.00000
Epoch : 57200, loss : 0.00000
Epoch : 57300, loss : 0.00000
Epoch : 57400, loss : 0.00000
Epoch : 57500, loss : 0.00000
Epoch : 57600, loss : 0.00000
Epoch : 57700, loss : 0.00000
Epoch : 57800, loss : 0.00000
Epoch : 57900, loss : 0.00000
Epoch : 58000, loss : 0.00000
Epoch : 58100, loss : 0.00000
Epoch : 58200, loss : 0.00000
Epoch : 58300, loss : 0.00001
Epoch : 58

Epoch : 82500, loss : 0.00000
Epoch : 82600, loss : 0.00000
Epoch : 82700, loss : 0.00000
Epoch : 82800, loss : 0.00000
Epoch : 82900, loss : 0.00000
Epoch : 83000, loss : 0.00000
Epoch : 83100, loss : 0.00000
Epoch : 83200, loss : 0.00000
Epoch : 83300, loss : 0.00000
Epoch : 83400, loss : 0.00000
Epoch : 83500, loss : 0.00000
Epoch : 83600, loss : 0.00000
Epoch : 83700, loss : 0.00000
Epoch : 83800, loss : 0.00000
Epoch : 83900, loss : 0.00000
Epoch : 84000, loss : 0.00000
Epoch : 84100, loss : 0.00000
Epoch : 84200, loss : 0.00000
Epoch : 84300, loss : 0.00000
Epoch : 84400, loss : 0.00000
Epoch : 84500, loss : 0.00000
Epoch : 84600, loss : 0.00000
Epoch : 84700, loss : 0.00000
Epoch : 84800, loss : 0.00000
Epoch : 84900, loss : 0.00000
Epoch : 85000, loss : 0.00000
Epoch : 85100, loss : 0.00001
Epoch : 85200, loss : 0.00000
Epoch : 85300, loss : 0.00000
Epoch : 85400, loss : 0.00000
Epoch : 85500, loss : 0.00000
Epoch : 85600, loss : 0.00000
Epoch : 85700, loss : 0.00000
Epoch : 85

Epoch : 10200, loss : 0.08071
Epoch : 10300, loss : 0.08071
Epoch : 10400, loss : 0.08071
Epoch : 10500, loss : 0.08071
Epoch : 10600, loss : 0.08071
Epoch : 10700, loss : 0.08071
Epoch : 10800, loss : 0.08071
Epoch : 10900, loss : 0.08071
Epoch : 11000, loss : 0.08071
Epoch : 11100, loss : 0.08071
Epoch : 11200, loss : 0.08071
Epoch : 11300, loss : 0.08071
Epoch : 11400, loss : 0.08071
Epoch : 11500, loss : 0.08071
Epoch : 11600, loss : 0.08071
Epoch : 11700, loss : 0.08071
Epoch : 11800, loss : 0.08071
Epoch : 11900, loss : 0.08071
Epoch : 12000, loss : 0.08071
Epoch : 12100, loss : 0.08071
Epoch : 12200, loss : 0.08071
Epoch : 12300, loss : 0.08071
Epoch : 12400, loss : 0.08071
Epoch : 12500, loss : 0.08071
Epoch : 12600, loss : 0.08071
Epoch : 12700, loss : 0.08071
Epoch : 12800, loss : 0.08071
Epoch : 12900, loss : 0.08071
Epoch : 13000, loss : 0.08071
Epoch : 13100, loss : 0.08071
Epoch : 13200, loss : 0.08071
Epoch : 13300, loss : 0.08071
Epoch : 13400, loss : 0.08071
Epoch : 13

...

Epoch : 84800, loss : 0.00000
Epoch : 84900, loss : 0.00000
Epoch : 85000, loss : 0.00000
Epoch : 85100, loss : 0.00000
Epoch : 85200, loss : 0.00000
Epoch : 85300, loss : 0.00000
Epoch : 85400, loss : 0.00000
Epoch : 85500, loss : 0.00000
Epoch : 85600, loss : 0.00001
Epoch : 85700, loss : 0.00000
Epoch : 85800, loss : 0.00000
Epoch : 85900, loss : 0.00001
Epoch : 86000, loss : 0.00000
Epoch : 86100, loss : 0.00000
Epoch : 86200, loss : 0.00000
Epoch : 86300, loss : 0.00000
Epoch : 86400, loss : 0.00000
Epoch : 86500, loss : 0.00000
Epoch : 86600, loss : 0.00000
Epoch : 86700, loss : 0.00000
Epoch : 86800, loss : 0.00000
Epoch : 86900, loss : 0.00000
Epoch : 87000, loss : 0.00000
Epoch : 87100, loss : 0.00000
Epoch : 87200, loss : 0.00000
Epoch : 87300, loss : 0.00000
Epoch : 87400, loss : 0.00000
Epoch : 87500, loss : 0.00000
Epoch : 87600, loss : 0.00000
Epoch : 87700, loss : 0.00000
Epoch : 87800, loss : 0.00000
Epoch : 87900, loss : 0.00000
Epoch : 88000, loss : 0.00000
Epoch : 88

Epoch : 12500, loss : 0.00007
Epoch : 12600, loss : 0.00004
Epoch : 12700, loss : 0.00009
Epoch : 12800, loss : 0.00017
Epoch : 12900, loss : 0.00010
Epoch : 13000, loss : 0.00005
Epoch : 13100, loss : 0.00004
Epoch : 13200, loss : 0.00004
Epoch : 13300, loss : 0.00005
Epoch : 13400, loss : 0.00011
Epoch : 13500, loss : 0.00005
Epoch : 13600, loss : 0.00006
Epoch : 13700, loss : 0.00007
Epoch : 13800, loss : 0.00004
Epoch : 13900, loss : 0.00006
Epoch : 14000, loss : 0.00010
Epoch : 14100, loss : 0.00004
Epoch : 14200, loss : 0.00004
Epoch : 14300, loss : 0.00005
Epoch : 14400, loss : 0.00004
Epoch : 14500, loss : 0.00004
Epoch : 14600, loss : 0.00004
Epoch : 14700, loss : 0.00004
Epoch : 14800, loss : 0.00004
Epoch : 14900, loss : 0.00004
Epoch : 15000, loss : 0.00017
Epoch : 15100, loss : 0.00004
Epoch : 15200, loss : 0.00011
Epoch : 15300, loss : 0.00004
Epoch : 15400, loss : 0.00013
Epoch : 15500, loss : 0.00004
Epoch : 15600, loss : 0.00007
Epoch : 15700, loss : 0.00004
Epoch : 15

Epoch : 39900, loss : 0.00003
Epoch : 40000, loss : 0.00003
Epoch : 40100, loss : 0.00003
Epoch : 40200, loss : 0.00009
Epoch : 40300, loss : 0.00003
Epoch : 40400, loss : 0.00003
Epoch : 40500, loss : 0.00002
Epoch : 40600, loss : 0.00002
Epoch : 40700, loss : 0.00003
Epoch : 40800, loss : 0.00004
Epoch : 40900, loss : 0.00002
Epoch : 41000, loss : 0.00005
Epoch : 41100, loss : 0.00004
Epoch : 41200, loss : 0.00004
Epoch : 41300, loss : 0.00002
Epoch : 41400, loss : 0.00002
Epoch : 41500, loss : 0.00006
Epoch : 41600, loss : 0.00002
Epoch : 41700, loss : 0.00006
Epoch : 41800, loss : 0.00003
Epoch : 41900, loss : 0.00003
Epoch : 42000, loss : 0.00002
Epoch : 42100, loss : 0.00002
Epoch : 42200, loss : 0.00002
Epoch : 42300, loss : 0.00002
Epoch : 42400, loss : 0.00007
Epoch : 42500, loss : 0.00003
Epoch : 42600, loss : 0.00003
Epoch : 42700, loss : 0.00003
Epoch : 42800, loss : 0.00002
Epoch : 42900, loss : 0.00002
Epoch : 43000, loss : 0.00003
Epoch : 43100, loss : 0.00005
Epoch : 43

Epoch : 67300, loss : 0.00009
Epoch : 67400, loss : 0.00003
Epoch : 67500, loss : 0.00002
Epoch : 67600, loss : 0.00002
Epoch : 67700, loss : 0.00001
Epoch : 67800, loss : 0.00001
Epoch : 67900, loss : 0.00002
Epoch : 68000, loss : 0.00002
Epoch : 68100, loss : 0.00004
Epoch : 68200, loss : 0.00004
Epoch : 68300, loss : 0.00002
Epoch : 68400, loss : 0.00001
Epoch : 68500, loss : 0.00016
Epoch : 68600, loss : 0.00002
Epoch : 68700, loss : 0.00002
Epoch : 68800, loss : 0.00002
Epoch : 68900, loss : 0.00002
Epoch : 69000, loss : 0.00001
Epoch : 69100, loss : 0.00002
Epoch : 69200, loss : 0.00002
Epoch : 69300, loss : 0.00001
Epoch : 69400, loss : 0.00003
Epoch : 69500, loss : 0.00003
Epoch : 69600, loss : 0.00006
Epoch : 69700, loss : 0.00001
Epoch : 69800, loss : 0.00001
Epoch : 69900, loss : 0.00001
Epoch : 70000, loss : 0.00003
Epoch : 70100, loss : 0.00001
Epoch : 70200, loss : 0.00002
Epoch : 70300, loss : 0.00003
Epoch : 70400, loss : 0.00006
Epoch : 70500, loss : 0.00001
Epoch : 70

Epoch : 94700, loss : 0.00002
Epoch : 94800, loss : 0.00001
Epoch : 94900, loss : 0.00002
Epoch : 95000, loss : 0.00002
Epoch : 95100, loss : 0.00001
Epoch : 95200, loss : 0.00003
Epoch : 95300, loss : 0.00004
Epoch : 95400, loss : 0.00002
Epoch : 95500, loss : 0.00001
Epoch : 95600, loss : 0.00004
Epoch : 95700, loss : 0.00001
Epoch : 95800, loss : 0.00003
Epoch : 95900, loss : 0.00004
Epoch : 96000, loss : 0.00001
Epoch : 96100, loss : 0.00002
Epoch : 96200, loss : 0.00001
Epoch : 96300, loss : 0.00001
Epoch : 96400, loss : 0.00003
Epoch : 96500, loss : 0.00003
Epoch : 96600, loss : 0.00001
Epoch : 96700, loss : 0.00002
Epoch : 96800, loss : 0.00001
Epoch : 96900, loss : 0.00002
Epoch : 97000, loss : 0.00002
Epoch : 97100, loss : 0.00004
Epoch : 97200, loss : 0.00002
Epoch : 97300, loss : 0.00001
Epoch : 97400, loss : 0.00002
Epoch : 97500, loss : 0.00003
Epoch : 97600, loss : 0.00001
Epoch : 97700, loss : 0.00002
Epoch : 97800, loss : 0.00002
Epoch : 97900, loss : 0.00004
Epoch : 98

In [59]:
del result['test']
result
result.to_csv('평가지표.csv',encoding='cp949')

In [60]:
del predict['test']

predict.to_csv('예측치.csv',encoding='cp949')
predict

,광명시,평택시,동두천시,안산시,고양시
2022-01-01,106.85910,103.37447,103.74559,102.33230,108.10480
2022-02-01,107.44382,103.37447,96.53315,100.79714,107.75527
2022-03-01,102.71256,103.37447,96.60989,83.02484,100.37602
2022-04-01,93.22448,103.37447,94.26414,82.51912,93.09251
2022-05-01,85.92702,103.37447,90.85525,104.21382,80.09635
2022-06-01,102.80136,103.37447,90.13998,83.02484,93.04290


In [ ]:
plt.plot(predicted_cases, label='Predicted monthly price')
plt.legend();

In [ ]:
plt.figure(figsize=(100,50))
plt.plot(y_3,label='real')
plt.plot(preds_3, label='predict')
plt.xticks(rotation=90)


plt.legend()